Census Data Pipeline

In [ ]:
# installing necessary libraries and modules

! pip install pandas
! python -m pip install "pymongo[srv]"
! pip install mysql-connector-python

In [ ]:
# importing the requirements

import pandas as pd
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import mysql.connector

In [ ]:
# reading the CSV data file

df = pd.read_csv("census_2011.csv", header = 0)

Task 1: Rename the Column names

In [ ]:
# Renaming the columns

column_headers = [
    {'State name':'State/UT'},
    {'District name':'District'},
    {'Male_Literate':'Literate_Male'},
    {'Female_Literate':'Literate_Female'},
    {'Rural_Households':'Households_Rural'},
    {'Urban_Households':'Households_Urban'},
    {'Age_Group_0_29':'Young_and_Adult'},
    {'Age_Group_30_49':'Middle_Aged'},
    {'Age_Group_50':'Senior_Citizen'},
    {'Age not stated':'Age_Not_Stated'}
    ]

def rename_columns(headers,df):    
    for i in headers:
        df.rename(columns = i, inplace = True)

rename_columns(column_headers, df)

Task 2: Rename State/UT Names

In [ ]:
# Outer for loop to get the state names. Inner for loop to check if the word is "and" else capitalize the word

def Capitalization(df):
    for i in range(len(df)):
        x = df.loc[i,'State/UT'].lower().split()
        for j in range(len(x)):
            if x[j] != 'and':
                x[j] = x[j].capitalize()
        df.loc[i,'State/UT'] = " ".join(x)

Capitalization(df)

Task 3: New State/UT formation

In [ ]:
# Reading the txt file

with open('Telangana.txt', 'r') as telangana:
    telangana_districts = telangana.read()
telangana_districts = telangana_districts.split("\n")

In [ ]:
# setting the values as Telangana if the districts are in the telangana_districts list

def telangana(df):
    for i in range(len(df)):
        if df.loc[i,'District'] in telangana_districts:
            df.loc[i,'State/UT'] = "Telangana"

telangana(df)

In [ ]:
# setting the values as Ladakh if the districts are in the ladakh list

def ladakh(df):
    ladakh = ['Leh', 'Kargil']
    for district in ladakh:
        for i in range(len(df)):
            if district in df.loc[i,'District']:
                df.loc[i,'State/UT'] = "Ladakh"

ladakh(df)

Task 4: Find and process Missing Data

In [ ]:
# Finding and storing the Percentage of missing data before cleaning for each column in a separate dictionary.
def missingDataPercent(df) :
    missing_data = {}
    columns = df.columns
    for i in columns:
        count = 0
        count += len(list(filter(lambda x : x == True, list(df[i].isnull()))))
        missing_data[i] = round((count * 100)/len(df), 2)

    return missing_data

missing_data_before_cleaning = missingDataPercent(df)

In [ ]:
# Function to return if the cell contains null value or not 

def null_values_filter(df,i,column):
    return list(df.loc[[i],column].isnull())[0]

In [ ]:
# Filling up the Population column

def fill_population(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Population') == True:
            if null_values_filter(df,i,'Male') == null_values_filter(df,i,'Female') == False:
                df.loc[i,"Population"] = int(df.loc[i,"Male"] + df.loc[i, "Female"])
            elif null_values_filter(df,i,'Young_and_Adult') == null_values_filter(df,i,'Middle_Aged') == null_values_filter(df,i,"Senior_Citizen") == null_values_filter(df,i,'Age_Not_Stated') == False:
                df.loc[i,'Population'] = int(df.loc[i,"Young_and_Adult"] + df.loc[i,"Middle_Aged"] + df.loc[i, "Senior_Citizen"] + df.loc[i,"Age_Not_Stated"])
            elif null_values_filter(df,i,'Hindus') == null_values_filter(df,i,'Muslims') == null_values_filter(df,i,'Christians') == null_values_filter(df,i,'Sikhs') == null_values_filter(df,i,'Buddhists') == null_values_filter(df,i,'Jains') == null_values_filter(df,i,'Others_Religions') == null_values_filter(df,i,'Religion_Not_Stated') == False:
                df.loc[i,'Population'] = int(df.loc[i,'Hindus'] + df.loc[i,'Muslims'] + df.loc[i,'Christians'] + df.loc[i,'Sikhs'] + df.loc[i,'Buddhists'] + df.loc[i,'Jains'] + df.loc[i,'Others_Religions'] + df.loc[i,'Religion_Not_Stated'])
            elif null_values_filter(df,i,'Workers') == null_values_filter(df,i,'Non_Workers') == False:
                df.loc[i,"Population"] = int(df.loc[i,'Workers'] + df.loc[i,'Non_Workers'])

fill_population(df)

In [ ]:
# Filling up the Male population

def fill_male(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Male') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Female') == False:
                df.loc[i,"Male"] = int(df.loc[i,"Population"] - df.loc[i,"Female"])

fill_male(df)

In [ ]:
# Filling up the Female population

def fill_female(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Female') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Male') == False:
                df.loc[i,"Female"] = int(df.loc[i,"Population"] - df.loc[i,"Male"])

fill_female(df)

In [ ]:
# Filling up the Literate population

def fill_literate(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Literate') == True:
            if null_values_filter(df,i,'Literate_Male') == null_values_filter(df,i,'Literate_Female') == False:
                df.loc[i,"Literate"] = int(df.loc[i,"Literate_Male"] + df.loc[i,"Literate_Female"])

fill_literate(df)

In [ ]:
# Filling up the Literate Male population

def fill_literate_male(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Literate_Male') == True:
            if null_values_filter(df,i,'Literate') == null_values_filter(df,i,'Literate_Female') == False:
                df.loc[i,"Literate_Male"] = int(df.loc[i,"Literate"] - df.loc[i,"Literate_Female"])

fill_literate_male(df)

In [ ]:
# Filling up the Literate Female population

def fill_literate_female(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Literate_Female') == True:
            if null_values_filter(df,i,'Literate') == null_values_filter(df,i,'Literate_Male') == False:
                df.loc[i,"Literate_Female"] = int(df.loc[i,"Literate"] - df.loc[i,"Literate_Male"])

fill_literate_female(df)

In [ ]:
# Filling up the SC column

def fill_sc(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'SC') == True:
            if null_values_filter(df,i,'Male_SC') == null_values_filter(df,i,'Female_SC') == False:
                df.loc[i,"SC"] = int(df.loc[i,"Male_SC"] + df.loc[i,"Female_SC"])

fill_sc(df)

In [ ]:
# Filing up the Male_SC Column

def fill_male_sc(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Male_SC') == True:
            if null_values_filter(df,i,'SC') == null_values_filter(df,i,'Female_SC') == False:
                df.loc[i,"Male_SC"] = int(df.loc[i,"SC"] - df.loc[i,"Female_SC"])

fill_male_sc(df)

In [ ]:
# Filing up the Female_SC Column

def fill_female_sc(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Female_SC') == True:
            if null_values_filter(df,i,'SC') == null_values_filter(df,i,'Male_SC') == False:
                df.loc[i,'Female_SC'] = int(df.loc[i,'SC'] - df.loc[i,'Male_SC'])

fill_female_sc(df)

In [ ]:
# Filling up the ST column

def fill_st(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'ST') == True:
            if null_values_filter(df,i,'Male_ST') == null_values_filter(df,i,'Female_ST') == False:
                df.loc[i,"ST"] = int(df.loc[i,"Male_ST"] + df.loc[i,"Female_ST"])

fill_st(df)

In [ ]:
# Filing up the Male_ST Column

def fill_male_st(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Male_ST') == True:
            if null_values_filter(df,i,'ST') == null_values_filter(df,i,'Female_ST') == False:
                df.loc[i,"Male_ST"] = int(df.loc[i,"ST"] - df.loc[i,"Female_ST"])

fill_male_st(df)

In [ ]:
# Filing up the Female_ST Column

def fill_female_st(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Female_ST') == True:
            if null_values_filter(df,i,'ST') == null_values_filter(df,i,'Male_ST') == False:
                df.loc[i,'Female_ST'] = int(df.loc[i,'ST'] - df.loc[i,'Male_ST'])

fill_female_st(df)

In [ ]:
# Filling up the Workers column

def fill_workers(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Workers') == True:
            if null_values_filter(df,i,'Male_Workers') == null_values_filter(df,i,'Female_Workers') == False:
                df.loc[i,"Workers"] = int(df.loc[i,"Male_Workers"] + df.loc[i,"Female_Workers"])
            elif null_values_filter(df,i,'Main_Workers') == null_values_filter(df,i,'Marginal_Workers') == False:
                df.loc[i,"Workers"] = int(df.loc[i,"Main_Workers"] + df.loc[i,"Marginal_Workers"])
            elif null_values_filter(df,i,'Population') == null_values_filter(df,i,'Non_Workers') == False:
                df.loc[i,"Workers"] = int(df.loc[i,"Population"] - df.loc[i,"Non_Workers"])

fill_workers(df)

In [ ]:
# Filing up the Male_Workers Column

def fill_male_workers(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Male_Workers') == True:
            if null_values_filter(df,i,'Workers') == null_values_filter(df,i,'Female_Workers') == False:
                df.loc[i,"Male_Workers"] = int(df.loc[i,"Workers"] - df.loc[i,"Female_Workers"])

fill_male_workers(df)

In [ ]:
# Filing up the Female_Workers Column

def fill_female_workers(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Female_Workers') == True:
            if null_values_filter(df,i,'Workers') == null_values_filter(df,i,'Male_Workers') == False:
                df.loc[i,'Female_Workers'] = int(df.loc[i,'Workers'] - df.loc[i,'Male_Workers'])

fill_female_workers(df)

In [ ]:
# Filling up the Main_Workers column

def fill_main_workers(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Main_Workers') == True:
            if null_values_filter(df,i,'Workers') == null_values_filter(df,i,'Marginal_Workers') == False:
                df.loc[i,'Main_Workers'] = int(df.loc[i,'Workers'] - df.loc[i,'Marginal_Workers'])
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Non_Workers') == null_values_filter(df,i,'Marginal_Workers') == False:
                df.loc[i,"Main_Workers"] = int(df.loc[i,'Population'] - (df.loc[i,'Non_Workers'] + df.loc[i,'Marginal_Workers']))

fill_main_workers(df)

In [ ]:
# Filling up the Marginal_Workers column

def fill_marginal_workers(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Marginal_Workers') == True:
            if null_values_filter(df,i,'Workers') == null_values_filter(df,i,'Main_Workers') == False:
                df.loc[i,'Marginal_Workers'] = int(df.loc[i,'Workers'] - df.loc[i,'Main_Workers'])
            elif null_values_filter(df,i,'Population') == null_values_filter(df,i,'Non_Workers') == null_values_filter(df,i,'Main_Workers') == False:
                df.loc[i,'Marginal_Workers'] = int(df.loc[i,'Population'] - (df.loc[i,'Non_Workers'] + df.loc[i,'Main_Workers']))

fill_marginal_workers(df)

In [ ]:
# Filling up the Non_Workers column:

def fill_non_workers(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Non_Workers') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Workers') == False:
                df.loc[i,'Non_Workers'] = int(df.loc[i,'Population'] - df.loc[i,'Workers'])
            elif null_values_filter(df,i,'Population') == null_values_filter(df,i,'Main_Workers') == null_values_filter(df,i,'Marginal_Workers') == False:
                df.loc[i,'Non_Workers'] = int(df.loc[i,'Population'] - (df.loc[i,"Main_Workers"] + df.loc[i,"Marginal_Workers"]))

fill_non_workers(df)

In [ ]:
# Filling up the Cultivator_Workers column

def fill_cultivator_workers(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Cultivator_Workers') == True:
            if null_values_filter(df,i,'Workers') == null_values_filter(df,i,'Agricultural_Workers') == null_values_filter(df,i,'Household_Workers') == null_values_filter(df,i,'Other_Workers') == False:
                df.loc[i,"Cultivator_Workers"] = int(df.loc[i,'Workers'] - (df.loc[i,'Agricultural_Workers'] + df.loc[i,'Household_Workers'] + df.loc[i,'Other_Workers']))

fill_cultivator_workers(df)

In [ ]:
# Filling up the Agricultural_Workers column

def fill_agricultural_workers(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Agricultural_Workers') == True:
            if null_values_filter(df,i,'Workers') == null_values_filter(df,i,'Cultivator_Workers') == null_values_filter(df,i,'Household_Workers') == null_values_filter(df,i,'Other_Workers') == False:
                df.loc[i,"Agricultural_Workers"] = int(df.loc[i,'Workers'] - (df.loc[i,'Cultivator_Workers'] + df.loc[i,'Household_Workers'] + df.loc[i,'Other_Workers']))

fill_agricultural_workers(df)

In [ ]:
# Filling up the Household_Workers column

def fill_household_workers(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Household_Workers') == True:
            if null_values_filter(df,i,'Workers') == null_values_filter(df,i,'Cultivator_Workers') == null_values_filter(df,i,'Agricultural_Workers') == null_values_filter(df,i,'Other_Workers') == False:
                df.loc[i,"Household_Workers"] = int(df.loc[i,'Workers'] - (df.loc[i,'Cultivator_Workers'] + df.loc[i,'Agricultural_Workers'] + df.loc[i,'Other_Workers']))

fill_household_workers(df)

In [ ]:
# Filling up the Other_Workers column

def fill_other_workers(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Other_Workers') == True:
            if null_values_filter(df,i,'Workers') == null_values_filter(df,i,'Cultivator_Workers') == null_values_filter(df,i,'Agricultural_Workers') == null_values_filter(df,i,'Household_Workers') == False:
                df.loc[i,"Other_Workers"] = int(df.loc[i,'Workers'] - (df.loc[i,'Cultivator_Workers'] + df.loc[i,'Agricultural_Workers'] + df.loc[i,'Household_Workers']))

fill_other_workers(df)

In [ ]:
# Filling up the Hindus column

def fill_hindus(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Hindus') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Muslims') == null_values_filter(df,i,'Christians') == null_values_filter(df,i,'Sikhs') == null_values_filter(df,i,'Buddhists') == null_values_filter(df,i,'Jains') == null_values_filter(df,i,'Others_Religions') == null_values_filter(df,i,'Religion_Not_Stated') == False:
                df.loc[i,'Hindus'] = int(df.loc[i,'Population'] - (df.loc[i,'Muslims'] + df.loc[i,'Christians'] + df.loc[i,'Sikhs'] + df.loc[i,'Buddhists'] + df.loc[i,'Jains'] + df.loc[i,'Others_Religions'] + df.loc[i,'Religion_Not_Stated']))

fill_hindus(df)

In [ ]:
# Filling up the Muslims column

def fill_muslims(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Muslims') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Hindus') == null_values_filter(df,i,'Christians') == null_values_filter(df,i,'Sikhs') == null_values_filter(df,i,'Buddhists') == null_values_filter(df,i,'Jains') == null_values_filter(df,i,'Others_Religions') == null_values_filter(df,i,'Religion_Not_Stated') == False:
                df.loc[i,'Muslims'] = int(df.loc[i,'Population'] - (df.loc[i,'Hindus'] + df.loc[i,'Christians'] + df.loc[i,'Sikhs'] + df.loc[i,'Buddhists'] + df.loc[i,'Jains'] + df.loc[i,'Others_Religions'] + df.loc[i,'Religion_Not_Stated']))

fill_muslims(df)

In [ ]:
# Filling up the Christians column

def fill_christians(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Christians') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Hindus') == null_values_filter(df,i,'Muslims') == null_values_filter(df,i,'Sikhs') == null_values_filter(df,i,'Buddhists') == null_values_filter(df,i,'Jains') == null_values_filter(df,i,'Others_Religions') == null_values_filter(df,i,'Religion_Not_Stated') == False:
                df.loc[i,'Christians'] = int(df.loc[i,'Population'] - (df.loc[i,'Hindus'] + df.loc[i,'Muslims'] + df.loc[i,'Sikhs'] + df.loc[i,'Buddhists'] + df.loc[i,'Jains'] + df.loc[i,'Others_Religions'] + df.loc[i,'Religion_Not_Stated']))

fill_christians(df)

In [ ]:
# Filling up the Sikhs column

def fill_sikhs(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Sikhs') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Hindus') == null_values_filter(df,i,'Muslims') == null_values_filter(df,i,'Christians') == null_values_filter(df,i,'Buddhists') == null_values_filter(df,i,'Jains') == null_values_filter(df,i,'Others_Religions') == null_values_filter(df,i,'Religion_Not_Stated') == False:
                df.loc[i,'Sikhs'] = int(df.loc[i,'Population'] - (df.loc[i,'Hindus'] + df.loc[i,'Muslims'] + df.loc[i,'Christians'] + df.loc[i,'Buddhists'] + df.loc[i,'Jains'] + df.loc[i,'Others_Religions'] + df.loc[i,'Religion_Not_Stated']))

fill_sikhs(df)

In [ ]:
# Filling up the Buddhists column

def fill_buddhists(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Buddhists') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Hindus') == null_values_filter(df,i,'Muslims') == null_values_filter(df,i,'Christians') == null_values_filter(df,i,'Sikhs') == null_values_filter(df,i,'Jains') == null_values_filter(df,i,'Others_Religions') == null_values_filter(df,i,'Religion_Not_Stated') == False:
                df.loc[i,'Buddhists'] = int(df.loc[i,'Population'] - (df.loc[i,'Hindus'] + df.loc[i,'Muslims'] + df.loc[i,'Christians'] + df.loc[i,'Sikhs'] + df.loc[i,'Jains'] + df.loc[i,'Others_Religions'] + df.loc[i,'Religion_Not_Stated']))

fill_buddhists(df)

In [ ]:
# Filling up the Jains column

def fill_jains(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Jains') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Hindus') == null_values_filter(df,i,'Muslims') == null_values_filter(df,i,'Christians') == null_values_filter(df,i,'Sikhs') == null_values_filter(df,i,'Buddhists') == null_values_filter(df,i,'Others_Religions') == null_values_filter(df,i,'Religion_Not_Stated') == False:
                df.loc[i,'Jains'] = int(df.loc[i,'Population'] - (df.loc[i,'Hindus'] + df.loc[i,'Muslims'] + df.loc[i,'Christians'] + df.loc[i,'Sikhs'] + df.loc[i,'Buddhists'] + df.loc[i,'Others_Religions'] + df.loc[i,'Religion_Not_Stated']))

fill_jains(df)

In [ ]:
# Filling up the Others_Religions column

def fill_others_religions(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Others_Religions') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Hindus') == null_values_filter(df,i,'Muslims') == null_values_filter(df,i,'Christians') == null_values_filter(df,i,'Sikhs') == null_values_filter(df,i,'Buddhists') == null_values_filter(df,i,'Jains') == null_values_filter(df,i,'Religion_Not_Stated') == False:
                df.loc[i,'Others_Religions'] = int(df.loc[i,'Population'] - (df.loc[i,'Hindus'] + df.loc[i,'Muslims'] + df.loc[i,'Christians'] + df.loc[i,'Sikhs'] + df.loc[i,'Buddhists'] + df.loc[i,'Jains'] + df.loc[i,'Religion_Not_Stated']))

fill_others_religions(df)

In [ ]:
# Filling up the Religion_Not_Stated column

def fill_religion_not_stated(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Religion_Not_Stated') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Hindus') == null_values_filter(df,i,'Muslims') == null_values_filter(df,i,'Christians') == null_values_filter(df,i,'Sikhs') == null_values_filter(df,i,'Buddhists') == null_values_filter(df,i,'Jains') == null_values_filter(df,i,'Others_Religions') == False:
                df.loc[i,'Religion_Not_Stated'] = int(df.loc[i,'Population'] - (df.loc[i,'Hindus'] + df.loc[i,'Muslims'] + df.loc[i,'Christians'] + df.loc[i,'Sikhs'] + df.loc[i,'Buddhists'] + df.loc[i,'Jains'] + df.loc[i,'Others_Religions']))

fill_religion_not_stated(df)

In [ ]:
# Filling up the Households_Rural column

def fill_households_rural(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Households_Rural') == True:
            if null_values_filter(df,i,'Households') == null_values_filter(df,i,'Households_Urban') == False:
                df.loc[i,'Households_Rural'] = int(df.loc[i,"Households"] - df.loc[i,'Households_Urban'])

fill_households_rural(df)

In [ ]:
# Filling up the Households_Urban column

def fill_households_urban(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Households_Urban') == True:
            if null_values_filter(df,i,'Households') == null_values_filter(df,i,'Households_Rural') == False:
                df.loc[i,'Households_Urban'] = int(df.loc[i,"Households"] - df.loc[i,'Households_Rural'])

fill_households_urban(df)

In [ ]:
# Filling up the Households column

def fill_households(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Households') == True:
            if null_values_filter(df,i,'Households_Urban') == null_values_filter(df,i,'Households_Rural') == False:
                df.loc[i,'Households'] = int(df.loc[i,"Households_Urban"] + df.loc[i,'Households_Rural'])

fill_households(df)

In [ ]:
# Filling up the Literate Education column

def fill_literate_education(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Literate_Education') == True:
            if null_values_filter(df,i,'Below_Primary_Education') == null_values_filter(df,i,'Primary_Education') == null_values_filter(df,i,'Middle_Education') == null_values_filter(df,i,'Secondary_Education') == null_values_filter(df,i,'Higher_Education') == null_values_filter(df,i,'Graduate_Education') == null_values_filter(df,i,'Other_Education') == False:
                df.loc[i,'Literate_Education'] = df.loc[i,'Below_Primary_Education'] + df.loc[i,'Primary_Education'] + df.loc[i,'Middle_Education'] + df.loc[i,'Secondary_Education'] + df.loc[i,'Higher_Education'] + df.loc[i,'Graduate_Education'] + df.loc[i,'Other_Education']
            elif null_values_filter(df,i,'Illiterate_Education') == null_values_filter(df,i,'Total_Education') == False:
                df.loc[i,'Literate_Education'] = int(df.loc[i,'Total_Education'] - df.loc[i,'Illiterate_Education'])

fill_literate_education(df)

In [ ]:
# Filling up the Below_Primary_Education column

def fill_below_primary_education(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Below_Primary_Education') == True:
            if null_values_filter(df,i,'Literate_Education') == null_values_filter(df,i,'Primary_Education') == null_values_filter(df,i,'Middle_Education') == null_values_filter(df,i,'Secondary_Education') == null_values_filter(df,i,'Higher_Education') == null_values_filter(df,i,'Graduate_Education') == null_values_filter(df,i,'Other_Education') == False:
                df.loc[i,'Below_Primary_Education'] = int(df.loc[i,'Literate_Education'] - (df.loc[i,'Primary_Education'] + df.loc[i,'Middle_Education'] + df.loc[i,'Secondary_Education'] + df.loc[i,'Higher_Education'] + df.loc[i,'Graduate_Education'] + df.loc[i,'Other_Education']))

fill_below_primary_education(df)

In [ ]:
# Filling up the Primary_Education column

def fill_primary_education(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Primary_Education') == True:
            if null_values_filter(df,i,'Literate_Education') == null_values_filter(df,i,'Below_Primary_Education') == null_values_filter(df,i,'Middle_Education') == null_values_filter(df,i,'Secondary_Education') == null_values_filter(df,i,'Higher_Education') == null_values_filter(df,i,'Graduate_Education') == null_values_filter(df,i,'Other_Education') == False:
                df.loc[i,'Primary_Education'] = int(df.loc[i,'Literate_Education'] - (df.loc[i,'Below_Primary_Education'] + df.loc[i,'Middle_Education'] + df.loc[i,'Secondary_Education'] + df.loc[i,'Higher_Education'] + df.loc[i,'Graduate_Education'] + df.loc[i,'Other_Education']))

fill_primary_education(df)

In [ ]:
# Filling up the Middle_Education column

def fill_middle_education(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Middle_Education') == True:
            if null_values_filter(df,i,'Literate_Education') == null_values_filter(df,i,'Below_Primary_Education') == null_values_filter(df,i,'Primary_Education') == null_values_filter(df,i,'Secondary_Education') == null_values_filter(df,i,'Higher_Education') == null_values_filter(df,i,'Graduate_Education') == null_values_filter(df,i,'Other_Education') == False:
                df.loc[i,'Middle_Education'] = int(df.loc[i,'Literate_Education'] - (df.loc[i,'Below_Primary_Education'] + df.loc[i,'Primary_Education'] + df.loc[i,'Secondary_Education'] + df.loc[i,'Higher_Education'] + df.loc[i,'Graduate_Education'] + df.loc[i,'Other_Education']))

fill_middle_education(df)

In [ ]:
# Filling up the Secondary_Education column

def fill_secondary_education(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Secondary_Education') == True:
            if null_values_filter(df,i,'Literate_Education') == null_values_filter(df,i,'Below_Primary_Education') == null_values_filter(df,i,'Primary_Education') == null_values_filter(df,i,'Middle_Education') == null_values_filter(df,i,'Higher_Education') == null_values_filter(df,i,'Graduate_Education') == null_values_filter(df,i,'Other_Education') == False:
                df.loc[i,'Secondary_Education'] = int(df.loc[i,'Literate_Education'] - (df.loc[i,'Below_Primary_Education'] + df.loc[i,'Primary_Education'] + df.loc[i,'Middle_Education'] + df.loc[i,'Higher_Education'] + df.loc[i,'Graduate_Education'] + df.loc[i,'Other_Education']))

fill_secondary_education(df)

In [ ]:
# Filling up the Higher_Education column

def fill_higher_education(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Higher_Education') == True:
            if null_values_filter(df,i,'Literate_Education') == null_values_filter(df,i,'Below_Primary_Education') == null_values_filter(df,i,'Primary_Education') == null_values_filter(df,i,'Middle_Education') == null_values_filter(df,i,'Secondary_Education') == null_values_filter(df,i,'Graduate_Education') == null_values_filter(df,i,'Other_Education') == False:
                df.loc[i,'Higher_Education'] = int(df.loc[i,'Literate_Education'] - (df.loc[i,'Below_Primary_Education'] + df.loc[i,'Primary_Education'] + df.loc[i,'Middle_Education'] + df.loc[i,'Secondary_Education'] + df.loc[i,'Graduate_Education'] + df.loc[i,'Other_Education']))

fill_higher_education(df)

In [ ]:
# Filling up the Graduate_Education column

def fill_graduate_education(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Graduate_Education') == True:
            if null_values_filter(df,i,'Literate_Education') == null_values_filter(df,i,'Below_Primary_Education') == null_values_filter(df,i,'Primary_Education') == null_values_filter(df,i,'Middle_Education') == null_values_filter(df,i,'Secondary_Education') == null_values_filter(df,i,'Higher_Education') == null_values_filter(df,i,'Other_Education') == False:
                df.loc[i,'Graduate_Education'] = int(df.loc[i,'Literate_Education'] - (df.loc[i,'Below_Primary_Education'] + df.loc[i,'Primary_Education'] + df.loc[i,'Middle_Education'] + df.loc[i,'Secondary_Education'] + df.loc[i,'Higher_Education'] + df.loc[i,'Other_Education']))

fill_graduate_education(df)

In [ ]:
# Filling up the Other_Education column

def fill_other_education(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Other_Education') == True:
            if null_values_filter(df,i,'Literate_Education') == null_values_filter(df,i,'Below_Primary_Education') == null_values_filter(df,i,'Primary_Education') == null_values_filter(df,i,'Middle_Education') == null_values_filter(df,i,'Secondary_Education') == null_values_filter(df,i,'Higher_Education') == null_values_filter(df,i,'Graduate_Education') == False:
                df.loc[i,'Other_Education'] = int(df.loc[i,'Literate_Education'] - (df.loc[i,'Below_Primary_Education'] + df.loc[i,'Primary_Education'] + df.loc[i,'Middle_Education'] + df.loc[i,'Secondary_Education'] + df.loc[i,'Higher_Education'] + df.loc[i,'Graduate_Education']))

fill_other_education(df)

In [ ]:
# Filling up the Illiterate_Education column

def fill_illiterate_education(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Illiterate_Education') == True:
            if null_values_filter(df,i,'Literate_Education') == null_values_filter(df,i,'Total_Education') == False:
                df.loc[i,'Illiterate_Education'] = int(df.loc[i,'Total_Education'] - df.loc[i,'Literate_Education'])

fill_illiterate_education(df)

In [ ]:
# Filling up the Total_Education column

def fill_total_education(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Total_Education') == True:
            if null_values_filter(df,i,'Literate_Education') == null_values_filter(df,i,'Illiterate_Education') == False:
                df.loc[i,'Total_Education'] = int(df.loc[i,'Literate_Education'] + df.loc[i,'Illiterate_Education'])

fill_total_education(df)

In [ ]:
# Filling up the Young_and_Adult column

def fill_young_and_adult(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Young_and_Adult') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Middle_Aged') == null_values_filter(df,i,'Senior_Citizen') == null_values_filter(df,i,'Age_Not_Stated') == False:
                df.loc[i,'Young_and_Adult'] = int(df.loc[i,'Population'] - (df.loc[i,'Middle_Aged'] + df.loc[i,'Senior_Citizen'] + df.loc[i,'Age_Not_Stated']))

fill_young_and_adult(df)

In [ ]:
# Filling up the Middle_Aged column

def fill_middle_aged(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Middle_Aged') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Young_and_Adult') == null_values_filter(df,i,'Senior_Citizen') == null_values_filter(df,i,'Age_Not_Stated') == False:
                df.loc[i,'Middle_Aged'] = int(df.loc[i,'Population'] - (df.loc[i,'Young_and_Adult'] + df.loc[i,'Senior_Citizen'] + df.loc[i,'Age_Not_Stated']))

fill_middle_aged(df)

In [ ]:
# Filling up the Senior_Citizen column

def fill_senior_citizen(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Senior_Citizen') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Young_and_Adult') == null_values_filter(df,i,'Middle_Aged') == null_values_filter(df,i,'Age_Not_Stated') == False:
                df.loc[i,'Senior_Citizen'] = int(df.loc[i,'Population'] - (df.loc[i,'Young_and_Adult'] + df.loc[i,'Middle_Aged'] + df.loc[i,'Age_Not_Stated']))

fill_senior_citizen(df)

In [ ]:
# Filling up the Senior_Citizen column

def fill_age_not_stated(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Age_Not_Stated') == True:
            if null_values_filter(df,i,'Population') == null_values_filter(df,i,'Young_and_Adult') == null_values_filter(df,i,'Middle_Aged') == null_values_filter(df,i,'Senior_Citizen') == False:
                df.loc[i,'Age_Not_Stated'] = int(df.loc[i,'Population'] - (df.loc[i,'Young_and_Adult'] + df.loc[i,'Middle_Aged'] + df.loc[i,'Senior_Citizen']))

fill_age_not_stated(df)

In [ ]:
# Filling up the Household_size_1_person_Households column

def fill_household_size_1_person_households(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Household_size_1_person_Households') == True:
            if null_values_filter(df,i,'Household_size_1_to_2_persons') == null_values_filter(df,i,'Household_size_2_persons_Households') == False:
                df.loc[i,'Household_size_1_person_Households'] = int(df.loc[i,'Household_size_1_to_2_persons'] - df.loc[i,'Household_size_2_persons_Households'])

fill_household_size_1_person_households(df)

In [ ]:
# Filling up the Household_size_2_persons_Households column

def fill_household_size_2_persons_households(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Household_size_2_persons_Households') == True:
            if null_values_filter(df,i,'Household_size_1_to_2_persons') == null_values_filter(df,i,'Household_size_1_person_Households') == False:
                df.loc[i,'Household_size_2_persons_Households'] = int(df.loc[i,'Household_size_1_to_2_persons'] - df.loc[i,'Household_size_1_person_Households'])

fill_household_size_2_persons_households(df)

In [ ]:
# Filling up the Household_size_1_to_2_persons column

def fill_household_size_1_to_2_persons(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Household_size_1_to_2_persons') == True:
            if null_values_filter(df,i,'Household_size_1_person_Households') == null_values_filter(df,i,'Household_size_2_persons_Households') == False:
                df.loc[i,'Household_size_1_to_2_persons'] = int(df.loc[i,'Household_size_1_person_Households'] + df.loc[i,'Household_size_2_persons_Households'])

fill_household_size_1_to_2_persons(df)

In [ ]:
# Filling up the Household_size_3_persons_Households column

def fill_household_size_3_persons_households(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Household_size_3_persons_Households') == True:
            if null_values_filter(df,i,'Household_size_4_persons_Households') ==  null_values_filter(df,i,'Household_size_5_persons_Households') == null_values_filter(df,i,'Household_size_3_to_5_persons_Households') == False:
                df.loc[i,'Household_size_3_persons_Households'] = int(df.loc[i,'Household_size_3_to_5_persons_Households'] - (df.loc[i,'Household_size_4_persons_Households'] + df.loc[i,'Household_size_5_persons_Households']))

fill_household_size_3_persons_households(df)

In [ ]:
# Filling up the Household_size_4_persons_Households column

def fill_household_size_4_persons_households(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Household_size_4_persons_Households') == True:
            if null_values_filter(df,i,'Household_size_3_persons_Households') ==  null_values_filter(df,i,'Household_size_5_persons_Households') == null_values_filter(df,i,'Household_size_3_to_5_persons_Households') == False:
                df.loc[i,'Household_size_4_persons_Households'] = int(df.loc[i,'Household_size_3_to_5_persons_Households'] - (df.loc[i,'Household_size_3_persons_Households'] + df.loc[i,'Household_size_5_persons_Households']))

fill_household_size_4_persons_households(df)

In [ ]:
# Filling up the Household_size_5_persons_Households column

def fill_household_size_5_persons_households(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Household_size_5_persons_Households') == True:
            if null_values_filter(df,i,'Household_size_3_persons_Households') ==  null_values_filter(df,i,'Household_size_4_persons_Households') == null_values_filter(df,i,'Household_size_3_to_5_persons_Households') == False:
                df.loc[i,'Household_size_5_persons_Households'] = int(df.loc[i,'Household_size_3_to_5_persons_Households'] - (df.loc[i,'Household_size_3_persons_Households'] + df.loc[i,'Household_size_4_persons_Households']))

fill_household_size_5_persons_households(df)

In [ ]:
# Filling up the Household_size_3_to_5_persons_Households column

def fill_household_size_3_to_5_persons_households(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Household_size_3_to_5_persons_Households') == True:
            if null_values_filter(df,i,'Household_size_3_persons_Households') == null_values_filter(df,i,'Household_size_4_persons_Households') == null_values_filter(df,i,'Household_size_5_persons_Households') == False:
                df.loc[i,'Household_size_3_to_5_persons_Households'] = int(df.loc[i,'Household_size_3_persons_Households'] + df.loc[i,'Household_size_4_persons_Households'] + df.loc[i,'Household_size_5_persons_Households'])

fill_household_size_3_to_5_persons_households(df)

In [ ]:
# Filling up the Power_Parity_Rs_45000_90000 column

def fill_power_parity_rs_45000_90000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == True:
            if null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_45000_150000') == False:
                df.loc[i,'Power_Parity_Rs_45000_90000'] = int(df.loc[i,'Power_Parity_Rs_45000_150000'] - df.loc[i,'Power_Parity_Rs_90000_150000'])
            elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Power_Parity_Rs_45000_90000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_90000_150000'] + df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_240000_330000'] + df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Rs_425000_545000'] + df.loc[i,'Power_Parity_Above_Rs_545000']))

fill_power_parity_rs_45000_90000(df)

In [ ]:
# Filling up the Power_Parity_Rs_90000_150000 column

def fill_power_parity_rs_90000_150000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == True:
            if null_values_filter(df,i,'Power_Parity_Rs_45000_150000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == False:
                df.loc[i,'Power_Parity_Rs_90000_150000'] = int(df.loc[i,'Power_Parity_Rs_45000_150000'] - df.loc[i,'Power_Parity_Rs_45000_90000'])
            elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Power_Parity_Rs_90000_150000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_240000_330000'] + df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Rs_425000_545000'] + df.loc[i,'Power_Parity_Above_Rs_545000']))

fill_power_parity_rs_90000_150000(df)

In [ ]:
# filling up the Power_Parity_Rs_45000_150000 column

def fill_power_parity_rs_45000_150000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Rs_45000_150000') == True:
            if null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == False:
                df.loc[i,'Power_Parity_Rs_45000_150000'] = int(df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_90000_150000'])
            elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Power_Parity_Rs_45000_150000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_240000_330000'] + df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Rs_425000_545000'] + df.loc[i,'Power_Parity_Above_Rs_545000']))

fill_power_parity_rs_45000_150000(df)

In [ ]:
# filling up the Power_Parity_Rs_150000_240000 column

def fill_power_parity_rs_150000_240000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == True:
            if null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_150000_330000') == False:
                df.loc[i,'Power_Parity_Rs_150000_240000'] = int(df.loc[i,'Power_Parity_Rs_150000_330000'] - df.loc[i,'Power_Parity_Rs_240000_330000'])
            elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Power_Parity_Rs_150000_240000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_90000_150000'] + df.loc[i,'Power_Parity_Rs_240000_330000'] + df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Rs_425000_545000'] + df.loc[i,'Power_Parity_Above_Rs_545000']))

fill_power_parity_rs_150000_240000(df)

In [ ]:
# filling up the Power_Parity_Rs_240000_330000 column

def fill_power_parity_rs_240000_330000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == True:
            if null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_150000_330000') == False:
                df.loc[i,'Power_Parity_Rs_240000_330000'] = int(df.loc[i,'Power_Parity_Rs_150000_330000'] - df.loc[i,'Power_Parity_Rs_150000_240000'])
            elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Power_Parity_Rs_240000_330000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_90000_150000'] + df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Rs_425000_545000'] + df.loc[i,'Power_Parity_Above_Rs_545000']))

fill_power_parity_rs_240000_330000(df)

In [ ]:
# filling up the Power_Parity_Rs_150000_330000 column

def fill_power_parity_rs_150000_330000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Rs_150000_330000') == True:
            if null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == False:
                df.loc[i,'Power_Parity_Rs_150000_330000'] = int(df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_240000_330000'])
            elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Power_Parity_Rs_150000_330000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_90000_150000'] + df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Rs_425000_545000'] + df.loc[i,'Power_Parity_Above_Rs_545000']))

fill_power_parity_rs_150000_330000(df)

In [ ]:
# filling up the Power_Parity_Rs_330000_425000 column

def fill_power_parity_rs_330000_425000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == True:
            if null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Rs_330000_545000') == False:
                df.loc[i,'Power_Parity_Rs_330000_425000'] = int(df.loc[i,'Power_Parity_Rs_330000_545000'] - df.loc[i,'Power_Parity_Rs_425000_545000'])
            elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Power_Parity_Rs_330000_425000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_90000_150000'] + df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_240000_330000'] + df.loc[i,'Power_Parity_Rs_425000_545000'] + df.loc[i,'Power_Parity_Above_Rs_545000']))

fill_power_parity_rs_330000_425000(df)

In [ ]:
# filling up the Power_Parity_Rs_425000_545000 column

def fill_power_parity_rs_425000_545000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == True:
            if null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Rs_330000_545000') == False:
                df.loc[i,'Power_Parity_Rs_425000_545000'] = int(df.loc[i,'Power_Parity_Rs_330000_545000'] - df.loc[i,'Power_Parity_Rs_330000_425000'])
            elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Power_Parity_Rs_425000_545000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_90000_150000'] + df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_240000_330000'] + df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Above_Rs_545000']))

fill_power_parity_rs_425000_545000(df)

In [ ]:
# filling up the Power_Parity_Rs_330000_545000
    
def fill_power_parity_rs_330000_545000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Rs_330000_545000') == True:
            if null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == False:
                df.loc[i,'Power_Parity_Rs_330000_545000'] = int(df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Rs_425000_545000'])
        elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Power_Parity_Rs_330000_545000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_90000_150000'] + df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_240000_330000'] + df.loc[i,'Power_Parity_Above_Rs_545000']))

fill_power_parity_rs_330000_545000(df)

In [ ]:
# Filling up the Total_Power_Parity column

def fill_total_power_parity(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Total_Power_Parity') == True:
            if null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_150000') == null_values_filter(df,i,'Power_Parity_Rs_150000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_545000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Total_Power_Parity'] = int(df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_45000_150000'] + df.loc[i,'Power_Parity_Rs_150000_330000'] + df.loc[i, 'Power_Parity_Rs_330000_545000'] + df.loc[i,'Power_Parity_Above_Rs_545000'])
            elif null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Total_Power_Parity'] = int(df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_90000_150000'] + df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_240000_330000'] + df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Rs_425000_545000'] + df.loc[i,'Power_Parity_Above_Rs_545000'])

fill_total_power_parity(df)

In [ ]:
# Filling up the Power_Parity_Less_than_Rs_45000 column

def fill_power_parity_less_than_rs_45000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == True:
            if null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == null_values_filter(df,i,'Power_Parity_Rs_45000_150000') == null_values_filter(df,i,'Power_Parity_Rs_150000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_545000') == null_values_filter(df,i,'Total_Power_Parity') == False:
                df.loc[i,'Power_Parity_Less_than_Rs_45000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Above_Rs_545000'] + df.loc[i,'Power_Parity_Rs_45000_150000'] + df.loc[i,'Power_Parity_Rs_150000_330000'] + df.loc[i, 'Power_Parity_Rs_330000_545000']))
            elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == False:
                df.loc[i,'Power_Parity_Less_than_Rs_45000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_90000_150000'] + df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_240000_330000'] + df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Rs_425000_545000'] + df.loc[i,'Power_Parity_Above_Rs_545000']))

fill_power_parity_less_than_rs_45000(df)

In [ ]:
# Filling up the Power_Parity_Above_Rs_545000 column

def fill_power_parity_above_rs_545000(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Power_Parity_Above_Rs_545000') == True:
            if null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_150000') == null_values_filter(df,i,'Power_Parity_Rs_150000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_545000') == null_values_filter(df,i,'Total_Power_Parity') == False:
                df.loc[i,'Power_Parity_Above_Rs_545000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Less_than_Rs_45000'] + df.loc[i,'Power_Parity_Rs_45000_150000'] + df.loc[i,'Power_Parity_Rs_150000_330000'] + df.loc[i, 'Power_Parity_Rs_330000_545000']))
            elif null_values_filter(df,i,'Total_Power_Parity') == null_values_filter(df,i,'Power_Parity_Less_than_Rs_45000') == null_values_filter(df,i,'Power_Parity_Rs_45000_90000') == null_values_filter(df,i,'Power_Parity_Rs_90000_150000') == null_values_filter(df,i,'Power_Parity_Rs_150000_240000') == null_values_filter(df,i,'Power_Parity_Rs_240000_330000') == null_values_filter(df,i,'Power_Parity_Rs_330000_425000') == null_values_filter(df,i,'Power_Parity_Rs_425000_545000') == False:
                df.loc[i,'Power_Parity_Above_Rs_545000'] = int(df.loc[i,'Total_Power_Parity'] - (df.loc[i,'Power_Parity_Rs_45000_90000'] + df.loc[i,'Power_Parity_Rs_90000_150000'] + df.loc[i,'Power_Parity_Rs_150000_240000'] + df.loc[i,'Power_Parity_Rs_240000_330000'] + df.loc[i,'Power_Parity_Rs_330000_425000'] + df.loc[i,'Power_Parity_Rs_425000_545000'] + df.loc[i,'Power_Parity_Less_than_Rs_45000']))

fill_power_parity_above_rs_545000(df)

In [ ]:
# Finding the percentage of missing data after cleaning is done

missing_data_after_cleaning = missingDataPercent(df)

In [ ]:
# Creating a CSV file to store the missing data details to visualize it in the streamlit along with other queries

missing_data = {
    'Columns': df.columns, 
    'Missing_data_before_cleaning (%)' : missing_data_before_cleaning.values(), 
    'Missing_data_after_cleaning (%)' : missing_data_after_cleaning.values()
    }
missing_data_df = pd.DataFrame(missing_data)
missing_data_df.to_csv('missing_data.csv', header=True)

Task  5: Save Data to MongoDB

In [ ]:
# Connecting with the MongoDB LocalHost

uri = "Please enter your MongoDB Localhost connection string here"

def create_mongo_connection(uri):

    # Create a new client and connect to the server
    client = MongoClient(uri, server_api=ServerApi('1'))

    # Send a ping to confirm a successful connection
    try:
        client.admin.command('ping')
        print("Pinged your deployment. You successfully connected to MongoDB!")
    except Exception as e:
        print(e)

create_mongo_connection(uri)

In [ ]:
# Creating a DB and Collection

def mongo_create_db_collection(uri):
    client=MongoClient(f'{uri}').census_db
    collection = 'census'

    # Checking if the collection already exists
    if collection in client.list_collection_names():
        client[collection].drop()
        client=MongoClient(f'{uri}').census_db.census
    else:
        client=MongoClient(f"{uri}").census_db.census

    return client

client = mongo_create_db_collection(uri)

In [ ]:
# inserting data into MongoDB 

def mongo_insert(client,df):
    keys = list(df.columns)
    for i in range(len(df)):
        values = []
        for j in keys:
            if j not in ['State/UT','District']:
                try:
                    values.append(int(df.loc[i,j]))
                except:
                    values.append(None)
            else:
                values.append(df.loc[i,j])
        doc = dict(zip(keys,values))
        client.insert_one(doc)

mongo_insert(client,df)

Task 6: Database connection and data upload

In [ ]:
# Connecting to MySQL server Localhost

mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",)

print(mydb)
mycursor = mydb.cursor(buffered=True)

In [ ]:
# Creating Database and table.

"""
Manually giving the names for columns which are more than 65 characters in length as MySQL column names have certain constraints
over Column name's length and special character usage.
"""

def create_db_table(df):
    mycursor.execute('CREATE DATABASE IF NOT EXISTS Census_DB')

    columns = list()
    for i in df.columns:
        if i == 'District code':
            columns.append('District_Code INT PRIMARY KEY')
        elif i == "State/UT":
            columns.append('State_UT VARCHAR(50)')
        elif i == "District":
            columns.append('District VARCHAR(50)')
        elif i == "Households_with_TV_Computer_Laptop_Telephone_mobile_phone_and_Scooter_Car":
            columns.append("Households_TV_Computer_Laptop_Telephone_mobile_phone_Scooter_Car INT")
        elif i == "Type_of_latrine_facility_Night_soil_disposed_into_open_drain_Households":
            columns.append("Type_of_latrine_facility_Night_soil_disposed_into_open_drain INT")
        elif i == "Type_of_latrine_facility_Flush_pour_flush_latrine_connected_to_other_system_Households":
            columns.append("Type_of_latrine_Flush_pour_connected_to_other_system_Households INT")
        elif i == "Not_having_latrine_facility_within_the_premises_Alternative_source_Open_Households":
            columns.append("Not_having_latrine_within_premises_Other_source_Open_Households INT")
        elif i == "Main_source_of_drinking_water_Handpump_Tubewell_Borewell_Households":
            columns.append("Source_of_drinking_water_Handpump_Tubewell_Borewell_Households INT")
        elif i == "Main_source_of_drinking_water_Other_sources_Spring_River_Canal_Tank_Pond_Lake_Other_sources__Households":
            columns.append("Drinking_water_Spring_River_Canal_Tank_Pond_Lake_Other_Household INT")
        else:
            columns.append(f'{i} INT')

    mycursor.execute(f'CREATE OR REPLACE TABLE Census_DB.census_2011 ({", ".join(columns)})')
    mydb.commit()

create_db_table(df)

In [ ]:
# Getting the column names from the table:

def get_columns(table_name):
    mycursor.execute(f"DESCRIBE {table_name}")
    table_columns = []
    for i in mycursor:
        table_columns.append(i[0])

    return tuple(table_columns)

table_columns = get_columns('Census_DB.census_2011')

In [ ]:
# Function to insert data into DB

"""
Creating placeholders for the values as we have to insert "Null" values inside the table and SQL accepts 
string formatting in '%s' method. Using executemany to insert bulk data into table.
"""

def db_insert_data(mongo_client, df, table_columns):
    placeholders = ", ".join(["%s"] * len(table_columns))
    columns = ", ".join(table_columns)

    data = []

    for i in range(1,len(df)+1):
        for j in mongo_client.find({'District code':i}):
            data.append(tuple(j.values())[1:])

    try :
        query = f'INSERT INTO Census_DB.Census_2011 ({columns}) VALUES ({placeholders})'
        mycursor.executemany(query, data)
    except Exception as e:
        print(e)

    mydb.commit()

db_insert_data(client, df, table_columns)

Please refer the "Census2011_Queries.py" for querying and displaying the results in streamlit.